In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [61]:
# Chat 모델 및 프롬프트 설정
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector    


# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature= 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()] 
)   

# Few-Shot Learning을 위한 예제 데이터
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 커스텀 Example Selector 구현
# BaseExampleSelector를 상속받아 직접 만든 선택기
# 랜덤 선택 방식 구현
class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        """생성자: 예제 리스트를 받아서 저장"""
        self.examples = examples

    def add_example(self, example): 
        """
        반드시 구현해야 하는 메서드
        """
        self.examples.append(example)

    def select_examples(self, input_variables):
        """
        반드시 구현해야 하는 메서드
        예제를 선택하는 메서드
        - input_variables: 사용자 입력 (예: {"country": "Brazil"})
        - 반환: 선택된 예제들의 리스트

        리스트를 반환하는 이유
        - FewShotPromptTemplate은 여러 개의 예제를 받을 수 있도록 설계됨
        - 1개만 선택해도 리스트 형태로 반환해야 함
        """
        from random import choice
        return [choice(self.examples)] # 무작위로 1개 선택해서 리스트로 반환


# 예제 형식 정의
example_prompt = PromptTemplate.from_template(
    "Human: {question} \nAI: {answer}"
)

# 커스텀 선택기 초기화
example_selector = RandomExampleSelector(
    examples = examples
)

# FewShotTemplate 구성
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector, # 커스텀 선택기 사용
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"]
)

# 실행할 때마다 다른 예제 선택됨
prompt.format(country = "Brazil")

'Human: What do you know about Italy? \nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'